# Competition — Network Generation

### Challenge Overview

Your goal is to generate a network that is as close as possible to the original real network. You do not have the original network in the explicit view, but you know some of its statistics. All statistics are in `stats.txt` file that contains a dictionary of the form
* number_nodes (number of nodes): value
* number_cc (number of connected components): value, sigma
* radius (radius of giant component): value, sigma
* diameter (diameter of giant component): value, sigma
* average_clustering (average clustering coefficient): value, sigma
* average_path_length (average path length): value, sigma
* degree_cdf (empirical CDF of degree distribution): values, probabilities

Meaning of all these sigmas is described in Evaluation section. 

You can use this code to draw CDF
```python
q_seq, p_seq = stats['degree_cdf']
    plt.plot(
        np.append(np.repeat(q_seq, 2)[1:], q_seq[-1]), 
        np.repeat(p_seq, 2)
    )
    plt.show()
```

### Evaluation Criteria

Your total score is calculated as weighted sum of 6 scores — similarities between statistics of original and generated networks. Each score takes values from the interval [0, 1], where 1 — absolute similarity with the original network. The scores are
* "KS"
    * 1 - KS_dist
    * where KS_dist is Kolmogorov-Smirnov test statistic value
* "Radius"
    * $\text{GK}(r, r', \sigma_r) = \exp\left[-\frac{(r - r')^2}{2\sigma_r^2}\right]$
    * where GK is Gaussian Kernel, $r$ is a radius of the original network, $r'$ is a radius of a generated network, $\sigma_r$ is a sigma of a radius from `stats.txt` file
* "Diameter", "Av. clustering", "Av. path length", "Number of CC" are calculated by Gaussian Kernel in the same way
* "Total"
    * 1/6 KS + 1/6 Radius + 1/6 Diameter + 1/6 Av. clustering + 1/6 Av. path length + 1/6 Number of CC

All scores immediately take value 0 if a generated network has incorrect number of nodes. All scores are multiplied by 100 on the leaderboard.

**Baselines**

Baselines are calculated by the following algorithm:
1. Generate a random degree sequence using Inverse Transform Sampling
2. Generate a valid graph by Configuration Model
3. Calculate total score
4. Repeat 1-3 steps 1000 times and accumulate a set of total scores

* Baseline for grade 4: beat a mean total score
* Baseline for grade 6: beat a mean + 3*sigma total score
* Baseline for grade 8: beat a maximum total score

Calculated baselines are in the leaderboard.

### Submission Guidelines

Submit a txt file with a list of edges without self-loops and parallels. The correct form is
```
1 2
1 3
3 2
```
and so on.

In [1]:
import pandas as pd
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline

In [7]:
with open('stats.txt', 'r') as file:
    data = eval(file.read())

In [8]:
def make_graph(data):
    G = nx.Graph()
    nodes = np.arange(data['number_nodes'])
    G.add_nodes_from(nodes)
    return G

In [9]:
g = make_graph(data)

In [10]:
df = pd.DataFrame(data).T
df.columns = ['value', 'sigma']
df

,value,sigma
number_nodes,1882,1882
radius,15,2
diameter,28,4
average_clustering,0.005067,0.001
average_path_length,11.748411,2.0
number_cc,168,32
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ..."


In [11]:
def calc_results(graph_list, df):
    for G in tqdm(graph_list):
        res = {}
        res['number_nodes'] = nx.number_of_nodes(G)
        res['number_cc'] = len(list(nx.connected_components(G)))
        largest_cc = G.subgraph(max(nx.connected_components(G), key=len))
        res['radius'] = nx.radius(largest_cc)
        res['diameter'] = nx.diameter(largest_cc)
        res['average_path_length'] = nx.average_shortest_path_length(largest_cc)
        res['average_clustering'] = nx.average_clustering(G)
        distr = pd.Series([x[1] for x in G.degree]).value_counts().sort_index()
        distr = distr.cumsum() / distr.sum()
        res['degree_cdf'] = distr
        res = pd.Series(res)
        df = pd.concat([df, res], axis = 1)
    return df

In [12]:
## test barabasi_albert_graph
g1 = nx.barabasi_albert_graph(data['number_nodes'], 1)
g2 = nx.barabasi_albert_graph(data['number_nodes'], 2)
g3 = nx.barabasi_albert_graph(data['number_nodes'], 3)
g4 = nx.barabasi_albert_graph(data['number_nodes'], 4)
calc_results([g1, g2, g3, g4], df)

  0%|          | 0/4 [00:00<?, ?it/s]

,value,sigma,0,0,0,0
number_nodes,1882,1882,1882,1882,1882,1882
radius,15,2,9,5,4,3
diameter,28,4,17,8,6,5
average_clustering,0.005067,0.001,0.0,0.017817,0.020764,0.027042
average_path_length,11.748411,2.0,6.64661,4.302005,3.743637,3.35915
number_cc,168,32,1,1,1,1
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ...",1 0.660999 2 0.837407 3 0.90967...,2 0.509564 3 0.708289 4 0.80658...,3 0.388417 4 0.597237 5 0.70882...,4 0.327311 5 0.520723 6 0.63708...


In [13]:
len(list(nx.connected_components(g)))

1882

In [14]:
import random

def random_barabasi(z):
    g = nx.Graph()
    n = 1882
    sizes = []
    m = data['number_cc'][0]
    for i in range(m - 1):
        k = random.randint(2, n - 2 * (m - i) - 1)
        sizes.append(k)
        n -= k

    sizes.append(n)


    for i in sizes: 
        n = nx.number_of_nodes(g)
        G = nx.barabasi_albert_graph(i, min(i-1, z))
        labels = {
            k:  n + k for k in range(i)
        }
        G = nx.relabel_nodes(G, labels)
        g = nx.union(G, g)
    return g

In [15]:
calc_results([g1, random_barabasi(1), random_barabasi(2)], df)

  0%|          | 0/3 [00:00<?, ?it/s]

,value,sigma,0,0,0
number_nodes,1882,1882,1882,1882,1882
radius,15,2,9,9,4
diameter,28,4,17,18,8
average_clustering,0.005067,0.001,0.0,0.0,0.053154
average_path_length,11.748411,2.0,6.64661,6.561129,4.019845
number_cc,168,32,1,168,168
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ...",1 0.660999 2 0.837407 3 0.90967...,1 0.724230 2 0.860786 3 0.917109 4...,1 0.170563 2 0.587673 3 0.74973...


In [16]:
with open('barabasi_albert_graph_1_168.txt', 'w') as f:
    for x in list(random_barabasi(1).edges()):
        f.write(f"{x[0]} {x[1]}\n")

In [17]:
def bar_rem(k, l):
    g = random_barabasi(k)
    for i in range(l):
        E = list(g.edges)
        e = random.choice(E)
        if (g.degree(e[0]) > 1) and (g.degree(e[1]) > 1):
            g.remove_edge(e[0], e[1])
            if len(list(nx.connected_components(g))) > 168:
                g.add_edge(e[0], e[1])
    return g

def bar_add(k, l):
    g = random_barabasi(k)
    for i in range(l):
        e = random.choice
        E = list(g.edges)
        e = random.choice(E)
        if (g.degree(e[0]) > 1) and (g.degree(e[1]) > 1):
            g.remove_edge(e[0], e[1])
            if len(list(nx.connected_components(g))) > 168:
                g.add_edge(e[0], e[1])
    return g

In [18]:
calc_results([random_barabasi(1), bar_rem(2, 3000), bar_rem(3, 4000), bar_rem(3, 4500), bar_rem(3, 5000)], df)

  0%|          | 0/5 [00:00<?, ?it/s]

,value,sigma,0,0,0,0,0
number_nodes,1882,1882,1882,1882,1882,1882,1882
radius,15,2,9,18,13,16,19
diameter,28,4,18,35,25,31,37
average_clustering,0.005067,0.001,0.0,0.002514,0.002224,0.001472,0.000187
average_path_length,11.748411,2.0,7.07835,13.054894,9.317036,10.280858,14.446823
number_cc,168,32,168,168,168,168,168
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ...",1 0.721041 2 0.856536 3 0.914984 4...,1 0.618491 2 0.845909 3 0.918172 4...,1 0.581296 2 0.811371 3 0.903826 4...,1 0.604145 2 0.836876 3 0.916047 4...,1 0.598831 2 0.846440 3 0.919235 4...


In [19]:
g0, g1, g2, g3 = bar_rem(2, 3000), bar_rem(2, 3100), bar_rem(2, 3250), bar_rem(2, 3400)
calc_results([g0, g1, g2, g3], df)

  0%|          | 0/4 [00:00<?, ?it/s]

,value,sigma,0,0,0,0
number_nodes,1882,1882,1882,1882,1882,1882
radius,15,2,15,15,13,15
diameter,28,4,29,30,26,30
average_clustering,0.005067,0.001,0.001968,0.000814,0.000585,0.000139
average_path_length,11.748411,2.0,10.290434,10.456919,9.856191,11.742754
number_cc,168,32,168,168,168,168
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ...",1 0.623804 2 0.837938 3 0.916578 4...,1 0.624867 2 0.845909 3 0.919235 4...,1 0.632306 2 0.841126 3 0.912859 4...,1 0.628055 2 0.846440 3 0.924548 4...


In [20]:
with open('bar_rem_graph_2_3100.txt', 'w') as f:
    for x in list(g1.edges()):
        f.write(f"{x[0]} {x[1]}\n")

In [47]:
from itertools import combinations


def increase_cc(G):
    H = G.copy()
    N = H.nodes
    cc = nx.average_clustering(H)
    while cc <= 0.0048:
        c = np.random.choice(N, 3, replace=False)
        triplet_score = np.sum([H.has_edge(*edge) for edge in list(combinations(c, 2))])
        if triplet_score == 2:
            for edge in list(combinations(c, 2)):
                if not H.has_edge(*edge):
                    H.add_edge(*edge)
                    cc = nx.average_clustering(H)
    return H
        
        
fixed_g1 = increase_cc(g1)

In [48]:
calc_results([g1, fixed_g1], df)

  0%|          | 0/2 [00:00<?, ?it/s]

,value,sigma,0,0
number_nodes,1882,1882,1882,1882
radius,15,2,15,15
diameter,28,4,30,29
average_clustering,0.005067,0.001,0.000814,0.005674
average_path_length,11.748411,2.0,10.456919,10.237608
number_cc,168,32,168,168
degree_cdf,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.6902231668437833, 0.8517534537725824, ...",1 0.624867 2 0.845909 3 0.919235 4...,1 0.622210 2 0.842189 3 0.918172 4...


In [50]:
with open('fixed_bar_rem_graph_2_3100.txt', 'w') as f:
    for x in list(fixed_g1.edges()):
        f.write(f"{x[0]} {x[1]}\n")